In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes trl gradio
import torch
import gc
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

print("Libraries installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 22.5 MB/s eta 0:00:00
Libraries installed successfully!


In [ ]:
# 1. Load Dataset
dataset_name = "ruslanmv/ai-medical-chatbot"
print(f"Loading medical dataset: {dataset_name}...")
dataset = load_dataset(dataset_name)

# 2. Load Tokenizer
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# 3. Format & Tokenize
def format_and_tokenize(sample):
    # Format: ### User: [symptom] ### Doctor: [diagnosis]
    text = f"### User:\n{sample['Patient']}\n\n### Doctor:\n{sample['Doctor']}"

    # Tokenize immediately to avoid errors later
    return tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=256 # Shorter length for speed
    )

print("Processing dataset (this takes about 1 minute)...")
# Map the function and REMOVE old text columns to prevent errors
tokenized_dataset = dataset["train"].map(
    format_and_tokenize,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# 4. Split Data (Train / Validation / Test)
# We use a small subset to make training fast for the assignment
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
final_dataset = split_dataset

print(f"Training Samples: {len(final_dataset['train'])}")
print(f"Validation Samples: {len(final_dataset['test'])}")

Loading medical dataset: ruslanmv/ai-medical-chatbot...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

dialogues.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Processing dataset (this takes about 1 minute)...


Map:   0%|          | 0/256916 [00:00<?, ? examples/s]

Training Samples: 59212
Validation Samples: 6580


In [ ]:
# 1. 4-Bit Quantization Config (To fit on Colab GPU)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 2. Load Base Model
print("Loading TinyLlama...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# 3. Apply LoRA (Low-Rank Adaptation)
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,               # Rank 16 (Higher precision for medical)
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
# ^ Screenshot this output for your report!

Loading TinyLlama...


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


In [ ]:
# --- STEP 1: INSTALLS (Run once) ---
!pip install -q transformers datasets peft accelerate bitsandbytes trl gradio

import torch
import gc
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

# --- STEP 2: LOAD TINYLLAMA (The Correct Model) ---
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
print(f"Loading Model: {model_id}...")

# Tokenizer Setup (Critical for Llama)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fixes the IndexError from before

# Model Setup (Quantized for Colab)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# Apply LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, peft_config)
# --- STEP 3: PREPARE MEDICAL DATA (FIXED) ---
print("Loading & Formatting Data...")
dataset = load_dataset("ruslanmv/ai-medical-chatbot")

def tokenize_function(examples):
    # When batched=True, 'examples' is a dictionary of lists:
    # {'Patient': ['symptom1', 'symptom2'], 'Doctor': ['diagnosis1', 'diagnosis2']}

    prompts = []
    # Iterate over the batch using zip to pair Patient and Doctor
    for patient, doctor in zip(examples['Patient'], examples['Doctor']):
        # Create the text prompt
        text = f"### User:\n{patient}\n\n### Doctor:\n{doctor}"
        prompts.append(text)

    # Tokenize the list of prompts all at once
    return tokenizer(
        prompts,
        padding="max_length",
        truncation=True,
        max_length=256
    )

# Use a small slice of data (1000 samples) to ensure it runs fast
train_data = dataset["train"].select(range(1000))

# Apply the fixed function
tokenized_data = train_data.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)

# Set format to PyTorch tensors
tokenized_data.set_format("torch")

# Split
split_data = tokenized_data.train_test_split(test_size=0.1)

from transformers import DataCollatorForLanguageModeling

# --- STEP 4: TRAINING LOOP (FIXED COLLATOR) ---
def run_training(lr, batch_size, run_name):
    print(f"\n--- Starting Experiment: {run_name} ---")
    torch.cuda.empty_cache()
    gc.collect()

    args = TrainingArguments(
        output_dir=f"./results/{run_name}",
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        max_steps=30,                # Short run
        learning_rate=lr,
        logging_steps=5,
        fp16=True,
        save_strategy="no",
        eval_strategy="no",
        report_to="none",
        remove_unused_columns=False  # Keep False
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=split_data["train"],
        # FIX: Use LanguageModeling collator. It auto-creates 'labels' for loss calculation.
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    )

    result = trainer.train()
    return result.metrics

# --- EXECUTE EXPERIMENTS ---
metrics_1 = run_training(lr=2e-4, batch_size=4, run_name="exp_1_baseline")
metrics_2 = run_training(lr=5e-5, batch_size=4, run_name="exp_2_low_lr")
metrics_3 = run_training(lr=2e-4, batch_size=8, run_name="exp_3_high_bs")

print("\n--- Final Results for Report ---")
print(f"Exp 1 Loss: {metrics_1['train_loss']}")
print(f"Exp 2 Loss: {metrics_2['train_loss']}")
print(f"Exp 3 Loss: {metrics_3['train_loss']}")

Loading Model: TinyLlama/TinyLlama-1.1B-Chat-v1.0...


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Loading & Formatting Data...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


--- Starting Experiment: exp_1_baseline ---


...


--- Starting Experiment: exp_2_low_lr ---


Step,Training Loss
5,2.281653
10,2.316611
15,2.348067
20,2.339478
25,2.318860
30,2.287147



--- Starting Experiment: exp_3_high_bs ---


Step,Training Loss
5,2.227280
10,2.223239
15,2.176899
20,2.130695
25,2.134766
30,2.127268



--- Final Results for Report ---
Exp 1 Loss: 2.489975833892822
Exp 2 Loss: 2.3153026580810545
Exp 3 Loss: 2.1700244267781574


In [ ]:
def ask_doctor(symptom):
    prompt = f"### User:\n{symptom}\n\n### Doctor:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=500,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Doctor:")[-1].strip()

# Test Case
print("Patient: I have a sharp pain in my chest.")
print(f"AI Doctor: {ask_doctor('I have a sharp pain in my chest when I breathe.')}")

Patient: I have a sharp pain in my chest.
AI Doctor: I have done a quick search online and found that you are suffering from a mild heart failure. This means


In [ ]:
import torch
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import subprocess

# --- STEP 1: LOAD MODEL (Auto-Recovery) ---
try:
    if 'model' not in globals() or model is None: raise NameError
except NameError:
    print("⏳ Reloading Brain (TinyLlama)... Please wait 45 seconds.")
    # Silent install if needed
    try: import bitsandbytes
    except ImportError: subprocess.run(["pip", "install", "-q", "transformers", "accelerate", "bitsandbytes", "gradio"])

    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
    print("✅ System Online!")

# --- STEP 2: INTELLIGENT MEDICAL BACKEND ---
def medical_chat(user_message, history, age, gender, symptoms):
    # If user checks boxes but types nothing, we auto-generate a message
    if not user_message and not symptoms:
        return "⚠️ Please describe your symptoms or select them from the checklist."

    if not user_message:
        user_message = f"I am a {age} year old {gender} experiencing symptoms."

    # 1. Context Construction
    context_str = f"Patient: {age}yr old {gender}."
    if symptoms:
        context_str += f" Symptoms: {', '.join(symptoms)}."

    # 2. Precision Prompt (Forces complete, structured answers)
    full_prompt = (
        f"### System:\nYou are an experienced ER Triage Nurse. "
        f"Review the patient profile: [{context_str}]. "
        f"You must answer in this exact format:\n"
        f"1. **Analysis**: What might be happening.\n"
        f"2. **Advice**: Immediate home care steps.\n"
        f"3. **Urgency**: When to see a doctor.\n"
        f"Keep it concise and do not stop mid-sentence.\n\n"
        f"### User:\n{user_message}\n\n"
        f"### Nurse:\n"
    )

    # 3. Generation
    inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.3,
            top_p=0.9,
            repetition_penalty=1.15
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Nurse:")[-1].strip()

# --- STEP 3: PROFESSIONAL UI (Fixed Theme) ---
theme = gr.themes.Soft(
    primary_hue="cyan",
    secondary_hue="blue",
    neutral_hue="slate",
).set(
    body_background_fill="#0f172a",    # Dark Blue Night Mode
    block_background_fill="#1e293b",   # Dark Slate Blocks
    block_border_width="0px",
    button_primary_background_fill="#06b6d4", # Neon Cyan Button
    button_primary_text_color="white",
    input_background_fill="#334155",
    # REMOVED the invalid 'text_color' line that caused the error
)

custom_css = """
.header-bar {
    background: linear-gradient(90deg, #06b6d4, #3b82f6);
    padding: 15px; border-radius: 10px; margin-bottom: 15px;
    text-align: center; color: white;
}
.warning-box {
    background: #450a0a; border: 1px solid #ef4444; color: #fca5a5;
    padding: 10px; border-radius: 5px; font-weight: bold; text-align: center;
}
/* Chat Bubbles */
.message-row.user-row .message-bubble {
    background-color: #06b6d4 !important; /* Cyan User */
    color: white !important;
}
.message-row.bot-row .message-bubble {
    background-color: #334155 !important; /* Dark Grey Bot */
    color: white !important;
    border: 1px solid #475569;
}
"""

with gr.Blocks(theme=theme, css=custom_css, title="MediBot Final") as demo:

    with gr.Column(elem_classes="header-bar"):
        gr.Markdown("# 🏥 MediBot Triage System v3.0")

    gr.HTML("""<div class="warning-box">⚠️ DISCLAIMER: AI Generative Model. Not a substitute for professional medical advice. Call 911 for emergencies.</div>""")

    with gr.Row():
        # --- LEFT PANEL (Inputs) ---
        with gr.Column(scale=1):
            gr.Markdown("### 📋 Patient Vitals")
            with gr.Row():
                age_in = gr.Number(label="Age", value=25, precision=0)
                gender_in = gr.Dropdown(["Male", "Female", "Other"], label="Gender", value="Female")

            gr.Markdown("### 🌡️ Symptoms")
            symptoms_in = gr.CheckboxGroup(
                ["High Fever", "Chest Pain", "Difficulty Breathing", "Nausea", "Headache", "Rash"],
                label="Check all that apply"
            )

            # Clickable Examples (Fixed: removed extra nesting)
            gr.Markdown("### ⚡ Quick Cases")

            def set_example(case):
                if case == "Headache": return 30, "Female", ["Headache"], "I have a throbbing headache and light sensitivity."
                if case == "Fever": return 8, "Male", ["High Fever", "Rash"], "My child has a fever of 103 and a rash."
                if case == "Chest": return 60, "Male", ["Chest Pain"], "Sharp pain in chest when breathing."

            with gr.Row():
                ex_btn1 = gr.Button("🤕 Headache Case")
                ex_btn2 = gr.Button("🤒 Child Fever")
                ex_btn3 = gr.Button("🫀 Chest Pain")

        # --- RIGHT PANEL (Chat) ---
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(label="Triage Log", avatar_images=["👤", "🩺"], height=550)

            with gr.Row():
                msg_in = gr.Textbox(placeholder="Describe condition here...", scale=4, lines=2)
                analyze_btn = gr.Button("Analyze ➤", variant="primary", scale=1)
                clear_btn = gr.Button("🗑️ Clear", variant="secondary", scale=1)

    # --- WIRING ---
    def respond(msg, hist, age, gend, symp):
        response = medical_chat(msg, hist, age, gend, symp)
        hist.append((msg if msg else "Selected Symptoms: " + ", ".join(symp), response))
        return "", hist

    # Example Buttons Logic
    ex_btn1.click(lambda: (30, "Female", ["Headache"], "I have a throbbing headache."), outputs=[age_in, gender_in, symptoms_in, msg_in])
    ex_btn2.click(lambda: (8, "Male", ["High Fever"], "My child has a fever of 103."), outputs=[age_in, gender_in, symptoms_in, msg_in])
    ex_btn3.click(lambda: (60, "Male", ["Chest Pain"], "Sharp pain in chest."), outputs=[age_in, gender_in, symptoms_in, msg_in])

    # Main Logic
    analyze_btn.click(respond, [msg_in, chatbot, age_in, gender_in, symptoms_in], [msg_in, chatbot])
    msg_in.submit(respond, [msg_in, chatbot, age_in, gender_in, symptoms_in], [msg_in, chatbot])
    clear_btn.click(lambda: (None, "", 25, "Female", []), outputs=[chatbot, msg_in, age_in, gender_in, symptoms_in])

demo.launch(share=True, debug=True)

⏳ Reloading Brain (TinyLlama)... Please wait 45 seconds.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ System Online!


/tmp/ipython-input-628427778.py:103: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, css=custom_css, title="MediBot Final") as demo:
/tmp/ipython-input-628427778.py:103: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, css=custom_css, title="MediBot Final") as demo:
/tmp/ipython-input-628427778.py:139: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Triage Log", avatar_images=["👤", "🩺"], height=550)
/tmp/ipython-input-628427778.py:139: 

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://51f9824a433fb01ada.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
